In [26]:
# Änderungen am Script werden nicht automatisch übernommen, daher: Neu laden!
import importlib
import model_trainer
importlib.reload(model_trainer)
from model_trainer import Trainer, TensorBoardLogger

from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim

In [27]:
batch_size = 64
learning_rate  = 0.01
epochs = 5

# Transformation: Flatten + Tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.reshape(-1))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [28]:
model = nn.Sequential( #nn.Flatten() hier obsolet, da in transform druchgeführt
    nn.Linear(28*28, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # -> default

In [29]:
logger = TensorBoardLogger(clear_old=True)
callbacks = [logger]

trainer = Trainer(
    model,
    train_loader,
    test_loader,
    criterion,
    optimizer,
    device="cpu",
    callbacks=callbacks
)

trainer.train(num_epochs=epochs)
trainer.evaluate()


TensorBoard Logs: runs/trial
Start Training: num_epochs=5
Epoch [1/5] Loss: 0.2219
Epoch [2/5] Loss: 0.1319
Epoch [3/5] Loss: 0.1127
Epoch [4/5] Loss: 0.0969
Epoch [5/5] Loss: 0.0925
#######
TensorBoard Logs: runs/trial
Start TensorBoard: tensorboard --logdir runs/trial
########
Accuracy: 96.57%


96.57

In [32]:
%reload_ext tensorboard
%tensorboard --logdir runs/trial